# Simple iteration for systems of linear equations

First, generate a random diagonally dominant matrix, for testing.

In [4]:
import numpy as np
rndm = np.random.RandomState(1234)

n = 10
A = rndm.uniform(size=(n, n)) + np.diagflat([15]*n)
b = rndm.uniform(size=n)
A

array([[1.51915195e+01, 6.22108771e-01, 4.37727739e-01, 7.85358584e-01,
        7.79975808e-01, 2.72592605e-01, 2.76464255e-01, 8.01872178e-01,
        9.58139354e-01, 8.75932635e-01],
       [3.57817270e-01, 1.55009951e+01, 6.83462935e-01, 7.12702027e-01,
        3.70250755e-01, 5.61196186e-01, 5.03083165e-01, 1.37684496e-02,
        7.72826622e-01, 8.82641191e-01],
       [3.64885984e-01, 6.15396178e-01, 1.50753812e+01, 3.68824006e-01,
        9.33140102e-01, 6.51378143e-01, 3.97202578e-01, 7.88730143e-01,
        3.16836122e-01, 5.68098653e-01],
       [8.69127390e-01, 4.36173424e-01, 8.02147642e-01, 1.51437668e+01,
        7.04260971e-01, 7.04581308e-01, 2.18792106e-01, 9.24867629e-01,
        4.42140755e-01, 9.09315959e-01],
       [5.98092228e-02, 1.84287084e-01, 4.73552788e-02, 6.74880944e-01,
        1.55946248e+01, 5.33310163e-01, 4.33240627e-02, 5.61433080e-01,
        3.29668446e-01, 5.02966833e-01],
       [1.11894318e-01, 6.07193706e-01, 5.65944643e-01, 6.76406199e-03,
   

# I.  Jacobi iteration

Given

$$
A x = b
$$

separate the diagonal part $D$,

$$ A = D + (A - D) $$

and write

$$
x = D^{-1} (D - A) x + D^{-1} b\;.
$$

Then iterate

$$
x_{n + 1} = B x_{n} + c\;,
$$

where 

$$
B = D^{-1} (A - D) \qquad \text{and} \qquad c = D^{-1} b
$$


Let's construct the matrix and the r.h.s. for the Jacobi iteration

In [6]:
diag_1d = np.diag(A)

B = -A.copy()
np.fill_diagonal(B, 0)

D = np.diag(diag_1d)
invD = np.diag(1./diag_1d)
BB = invD @ B 
c = invD @ b
B

array([[ 0.        , -0.62210877, -0.43772774, -0.78535858, -0.77997581,
        -0.27259261, -0.27646426, -0.80187218, -0.95813935, -0.87593263],
       [-0.35781727,  0.        , -0.68346294, -0.71270203, -0.37025075,
        -0.56119619, -0.50308317, -0.01376845, -0.77282662, -0.88264119],
       [-0.36488598, -0.61539618,  0.        , -0.36882401, -0.9331401 ,
        -0.65137814, -0.39720258, -0.78873014, -0.31683612, -0.56809865],
       [-0.86912739, -0.43617342, -0.80214764,  0.        , -0.70426097,
        -0.70458131, -0.21879211, -0.92486763, -0.44214076, -0.90931596],
       [-0.05980922, -0.18428708, -0.04735528, -0.67488094,  0.        ,
        -0.53331016, -0.04332406, -0.56143308, -0.32966845, -0.50296683],
       [-0.11189432, -0.60719371, -0.56594464, -0.00676406, -0.61744171,
         0.        , -0.79052413, -0.99208147, -0.95880176, -0.79196414],
       [-0.28525096, -0.62491671, -0.4780938 , -0.19567518, -0.38231745,
        -0.05387369,  0.        , -0.98200474

In [14]:
# sanity checks
from numpy.testing import assert_allclose

assert_allclose(-B + D, A)


# xx is a "ground truth" solution, compute it using a direct method
xx = np.linalg.solve(A, b)

np.testing.assert_allclose(A@xx, b)
np.testing.assert_allclose(D@xx, B@xx + b)
np.testing.assert_allclose(xx, BB@xx + c)
xx

array([ 0.03919429,  0.03780037,  0.04283232,  0.02365951,  0.05745031,
       -0.00030244, -0.00577279,  0.03177549, -0.00422849,  0.05284648])

Check that $\| B\| \leqslant 1$:

In [8]:
np.linalg.norm(BB)

0.36436161983015336

### Do the Jacobi iteration

In [9]:
n_iter = 50

x0 = np.ones(n)
x = x0
for _ in range(n_iter):
    x = BB @ x + c

In [10]:
# Check the result:

A @ x - b

array([ 0.00000000e+00,  2.22044605e-16,  0.00000000e+00, -1.11022302e-16,
        0.00000000e+00,  0.00000000e+00, -2.08166817e-17,  0.00000000e+00,
        0.00000000e+00,  2.22044605e-16])

### Task I.1

Collect the proof-of-concept above into a single function implementing the Jacobi iteration. This function should receive the r.h.s. matrix $A$, the l.h.s. vector `b`, and the number of iterations to perform.


The matrix $A$ in the illustration above is strongly diagonally dominant, by construction. 
What happens if the diagonal matrix elements of $A$ are made smaller? Check the convergence of the Jacobi iteration, and check the value of the norm of $B$.

(20% of the total grade)


In [20]:
# ... ENTER YOUR CODE HERE ...
def jacobi_iteration(x,A,b,iter_num=1000):
    #matrix D
    diag_1d = np.diag(A)
    D = np.diag(diag_1d)
    #inverse of D
    invD = np.diag(1./diag_1d)
    #matrix B
    B = -A.copy()
    np.fill_diagonal(B, 0)
    BB = invD @ B 
    #rhs vector
    c = invD @ b
    #iterate
    iters=1
    while iters<iter_num:
        x=np.dot(BB,x)+c
        iters=iters+1
    return x

x=jacobi_iteration(x0,A,b,iter_num=1000)   
x

array([ 0.03919429,  0.03780037,  0.04283232,  0.02365951,  0.05745031,
       -0.00030244, -0.00577279,  0.03177549, -0.00422849,  0.05284648])

# II. Seidel's iteration.

##### Task II.1

Implement the Seidel's iteration. 

Test it on a random matrix. Study the convergence of iterations, relate to the norm of the iteration matrix.

(30% of the total grade)

In [19]:
# ... ENTER YOUR CODE HERE ...
def seidel_iteration(x,A,b,iter_num=1000):
    #matrix U
    U=np.zeros_like(A)
    for i in range(A.shape[0]):
        for j in range(i+1,A.shape[0]):
            U[i][j]=A[i][j]
    LHS=A-U
    iters=0
    while iters<iter_num:
        x=np.dot(np.linalg.inv(LHS),b-np.dot(U,x))
        iters=iters+1
    return x
x=seidel_iteration(x0,A,b,iter_num=1000)   
x

array([ 0.03919429,  0.03780037,  0.04283232,  0.02365951,  0.05745031,
       -0.00030244, -0.00577279,  0.03177549, -0.00422849,  0.05284648])

# III. Minimum residual scheme

### Task III.1

Implement the $\textit{minimum residual}$ scheme: an explicit non-stationary method, where at each step you select the iteration parameter $\tau_n$ to minimize the residual $\mathbf{r}_{n+1}$ given $\mathbf{r}_n$. Test it on a random matrix, study the convergence to the solution, in terms of the norm of the residual and the deviation from the ground truth solution (which you can obtain using a direct method). Study how the iteration parameter $\tau_n$ changes as iterations progress.

(50% of the grade)

In [38]:
# ... ENTER YOUR CODE HERE ...
def mrm_iteration(x,A,b,iter_num=1000):
    iters=0
    x,b=x.reshape([-1,1]),b.reshape([-1,1])
    while iters<iter_num:
        #current residual
        r=(np.dot(A,x)-b).reshape([-1,1])
        #Ar
        Ar=np.dot(A,r)
        #current tau
        tau=np.sum(np.dot(r.T,Ar))/np.sum(Ar**2)
        #update
        x=x-tau*r
        iters=iters+1
    return x

In [39]:
mrm_iteration(x0,A,b,iter_num=1000)

array([[ 0.03919429],
       [ 0.03780037],
       [ 0.04283232],
       [ 0.02365951],
       [ 0.05745031],
       [-0.00030244],
       [-0.00577279],
       [ 0.03177549],
       [-0.00422849],
       [ 0.05284648]])